In [1]:
import captum
import torch
import utils
import model.net as net
from model.data_loader import DataLoader
from torch import device as dev
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization
from captum_viz import interpret_sequence

pred:  Extracellular ( 0.99 ) , delta:  8.579375077033546e-08
pred:  Plastid ( 1.00 ) , delta:  8.487881277829956e-07
pred:  Cytoplasm ( 0.88 ) , delta:  3.6450237450580403e-07
pred:  Mitochondrion ( 1.00 ) , delta:  2.3572954965800363e-07
pred:  Cytoplasm ( 0.82 ) , delta:  7.547191882562032e-08


In [2]:
MODEL_DIR = 'experiments/base_model/'
DATA_DIR = 'data/'
sentences_file = DATA_DIR + 'train/sentences.txt'
labels_file = DATA_DIR + 'train/labels.txt'
weights = MODEL_DIR + 'best.pth'
params = utils.Params(MODEL_DIR+'params.json')
params.vocab_size = 25
params.number_of_classes = 10
params.cuda = torch.cuda.is_available()

In [ ]:
loader = DataLoader(DATA_DIR, params)
data = loader.load_data(['train', 'val'], DATA_DIR)
train_data = data['train']
train_data_iterator = loader.data_iterator(train_data, params, shuffle=False)
train_batch, label_batch = next(train_data_iterator)

classes = ['Extracellular', 'Plastid', 'Cytoplasm', 'Mitochondrion', 
'Nucleus', 'ER', 'Golgi', 'Membrane', 'Lysosome', 'Peroxisome']

In [ ]:
sentences = open(sentences_file).read().splitlines()
labels = open(labels_file).read().splitlines()
token_reference = TokenReferenceBase(reference_token_idx=loader.pad_ind)
layer_ig = LayerIntegratedGradients(model, model.embedding)
vis_data_records = []

In [ ]:
model = net.Net(params).cuda() if params.cuda else net.Net(params)
checkpoint = torch.load(weights, map_location=dev('cpu'))
model.load_state_dict(checkpoint['state_dict'])
model.eval()

In [ ]:
interpret_sequence(model, sentences[:5], train_batch, vis_data_records)

In [ ]:
visualization.visualize_text(vis_data_records)

In [ ]:
vis_data_records